In [1]:
#Importing the libaries
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
#Making a request to the New York Times Bestsellers page
response = requests.get('https://www.nytimes.com/books/best-sellers/')

In [3]:
#Parsing the HTML
html = BeautifulSoup(response.content, 'html.parser')

#Creating a dictionary with the 
#category of the lists and the 
#name of the lists

nyt_lists = {}
list_sections = html.find_all('div', class_ = 'css-nl1pro')
for i, list_section in enumerate(list_sections):
    list_category = list_section.find('h2', class_ = 'css-ok8je7').text
    nyt_lists[list_category] = []
    
    #The 'Weekly Best Sellers Lists' has the 'li' tag 
    #but the 'Monthly Best Sellers Lists' has the 'div' tag
    #so extraction has to be handled differently
    if i == 0:
        nyt_list_names = list_section.find_all('li', class_ = 'css-108lmbq')
        for nyt_list in nyt_list_names:
            nyt_lists[list_category].append(nyt_list.text)
    else:
        nyt_list_names= list_section.find_all('div', class_ = 'css-108lmbq')
        for nyt_list in nyt_list_names:
            nyt_lists[list_category].append(nyt_list.text)

In [4]:
dataframes = []
#list names has the 
list_category = list(nyt_lists.keys())

for i, a_list in enumerate(list_category):
    temp_df = pd.DataFrame.from_dict(nyt_lists[a_list])
    temp_df['frequency'] = list_category[i]
    temp_df['frequency'] = temp_df['frequency'].str.lower().str.replace('best sellers lists', '').str.strip().str.capitalize()
    dataframes.append(temp_df)

#Creating a dataframe 
df = pd.concat(dataframes)
df.rename(columns = {0: 'list_name'}, inplace = True)
df.reset_index(drop = True, inplace = True)
df['list_name'] = df['list_name'].str.strip()
indices = [i for i in range(1, len(df) + 1)]
df['list_id'] = indices

#Reordering the columns for database loading
df = df[['list_id', 'list_name', 'frequency']]

#Creating a csv file of the NYT bestseller lsists 
df.to_csv('lists.csv', index = False, encoding = 'utf-8')